In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import re
import json


In [18]:
def collecte_products(data_collected,prods,sexe,request):
    for prod in prods: 
            txt = [p.text.split() for p in prod.find_all("h3")]
            prices = [p.text for p in prod.find_all("p")]
            finalprice = ""
            originprice = ""
            promo = ""
            if prices:
                    finalprice = ",".join(re.findall( r'\d+', prices[0]))
                    if len(prices) == 2 :
                        calcul = [p.text.split("-") for p in prod.find_all("p")][1]
                        originprice = ",".join(re.findall( r'\d+', calcul[0]))
                        if len(calcul) == 2 :
                            promo = ",".join(re.findall( r'\d+', calcul[1]))    

            if len(txt) == 2 :
                if originprice == "" :
                    originprice = finalprice
                    promo = 0
                data_collected.append({    "sexe" : sexe,
                                           "product": request,
                                           "mark":txt[0][0],
                                           "description":" ".join(txt[1]),
                                           "finalprice": finalprice,
                                           "originprice": originprice ,
                                           "promo": promo,
                                           "picture": [img["src"] for img in prod.find_all("img")]})
    return data_collected


def get_data_products (driver,request,sexe) :
    data_collected = []
    url = "https://www.zalando.fr/"+sexe+"/?q="+request
    print(url)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    max_page = int(soup.find("span","_0Qm8W1 _7Cm1F9 FxZV-M pVrzNP JCuRr_ _0xLoFW uEg2FS FCIprz").text.split()[-1])
    for page in range(1,60,1) :
        driver.get("https://www.zalando.fr/catalogue/?q="+request+ "&p=" + str(page))
        soup = BeautifulSoup(driver.page_source, 'lxml')
        prods = soup.find_all('article')
        collecte_products(data_collected,prods,sexe,request)
    path_json = "data_result_"+sexe+"_"+request+".json"
    with open(path_json, 'w') as f:
        json.dump(data_collected, f)
    return data_collected




In [ ]:
driver = webdriver.Firefox()
data_collected = get_data_products (driver,"tshirt","homme")

In [ ]:
driver = webdriver.Firefox()
data_collected = get_data_products (driver,"tshirt","femme")

In [ ]:
driver = webdriver.Firefox()
data_collected = get_data_products (driver,"jeans","homme")

https://www.zalando.fr/homme/?q=jeans


In [9]:
driver = webdriver.Firefox()
data_collected = get_data_products (driver,"jeans","femme")

https://www.zalando.fr/femme/?q=jeans


In [10]:
driver = webdriver.Firefox()
data_collected = get_data_products (driver,"pentalon","homme")

https://www.zalando.fr/homme/?q=pentalon


In [11]:
driver = webdriver.Firefox()
data_collected = get_data_products (driver,"pentalon","femme")

https://www.zalando.fr/femme/?q=pentalon


In [12]:
driver = webdriver.Firefox()
data_collected = get_data_products (driver,"chemise","homme")

https://www.zalando.fr/homme/?q=chemise


In [13]:
driver = webdriver.Firefox()
data_collected = get_data_products (driver,"chemise","femme")

https://www.zalando.fr/femme/?q=chemise


In [16]:
driver = webdriver.Firefox()
data_collected = get_data_products (driver,"short","homme")

https://www.zalando.fr/homme/?q=short


In [17]:
driver = webdriver.Firefox()
data_collected = get_data_products (driver,"short","femme")

https://www.zalando.fr/femme/?q=short


In [79]:
# 17734
d1 = pd.read_json("data_result_homme_tshirt.json")

# 3223
d2 = pd.read_json("data_result_femme_tshirt.json")

# 2617
d3 = pd.read_json("data_result_homme_pentalon.json")

# 1834
d4 = pd.read_json("data_result_femme_pentalon.json")

# 1923
d5 = pd.read_json("data_result_homme_chemise.json")

# 1965
d6 = pd.read_json("data_result_femme_chemise.json")

# 2459
d7 = pd.read_json("data_result_homme_short.json")

# 3294
d8 = pd.read_json("data_result_femme_short.json")

# 2113
d9 = pd.read_json("data_result_homme_jeans.json")

# 2042
d10 = pd.read_json("data_result_femme_jeans.json")

In [ ]:
df_result = pd.concat([d1,d2,d3,d4,d5,d6,d7,d8,d9,d10], axis=0)
df_result

In [8]:
# RESET INDEX OF ROWS
df_result = df_result.reset_index()


In [83]:
# Treat links
df_result['picture'] = df_result['picture'].apply(lambda x: x[0])


In [12]:
# Delete deplucate rows
df_result = df_result.drop_duplicates()


In [13]:
# Nombre de toutes les produits
len(df_result)

39204

In [17]:
# pre processing  
def treat_finalprice(x):
    x = x.replace("," , ".")
    float_regex = "[-+]?[0-9]*\.?[0-9]*"
    match = re.findall(float_regex, x)
    if match[0] != "":
        x = float(match[0])
    return x

def treating_prices(df):
    df['finalprice'] = df['finalprice'].apply(lambda x: treat_finalprice(x))
    df['originprice'] = df['originprice'].apply(lambda x: treat_finalprice(x))
    return df

def get_dataframe(path):
    df = pd.read_csv(path)
    df = df[['product', 'picture','sexe','mark', 'description','finalprice', 'originprice', 'promo',]]
    df = treating_prices(df)
    return df

In [18]:
path = "all_data_result_final.csv"
df_result = pd.read_csv(path)

In [4]:
# path = "all_data_result_final.csv"
# df.to_csv(path)